<a href="https://colab.research.google.com/github/yasmeenha/bigdata/blob/main/celaba_starter.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This notebook was the joint effort of Karthik Gandu, Mahesh Mudavat and Russel Pears. Use it as a starter for your own work in Project 2 of this course.

In [ ]:
# download the dataset from kaggle if you do not have a Kaggle account, you need to create one
# Use the Kaggle public API and output a token into your local drive ("Downloads" for windows PCs)

!pip install -q kaggle
from google.colab import files
files.upload() #Upload the token file containing  the token
# create a directory named kaggle and copy kaggle.json file to kaggle directory
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/

# change the permission of the file
!chmod 600 ~/.kaggle/kaggle.json

# download the dataset
!kaggle datasets download -d jessicali9530/celeba-dataset

# unzip the dataset to the current directory and remove the zip file
!unzip celeba-dataset.zip -d celeba-dataset && rm celeba-dataset.zip

KeyboardInterrupt: ignored

In [ ]:
# Importing needed libraries
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import seaborn as sns
import cv2
import random
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing import image
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.callbacks import ReduceLROnPlateau
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
import itertools
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline

In [ ]:
#  read in the annotations
anno = pd.read_csv('celeba-dataset/list_attr_celeba.csv') 
anno.head()

In [ ]:
# target only the required targets from the annotations data frame and discard the rest


In [ ]:
#  read in the landmarks
landmarks = pd.read_csv('celeba-dataset/list_landmarks_align_celeba.csv')
landmarks.head()

In [ ]:
# read in the partitions to discover train, validation and test segments
partition = pd.read_csv('celeba-dataset/list_eval_partition.csv')
partition.head()

In [ ]:
# read in the bounding boxes - these will be used to crop the images
bbox = pd.read_csv('celeba-dataset/list_bbox_celeba.csv')
bbox.head()

In [ ]:
# perform an outer join of the anno and bbox data frames on image_id

# perform an outer join of the result with the partition data frame on image_id to obtain integrated partitions


In [ ]:
# divide the integrated data frame into train, test and validation data frames based on the partition column 0-train, 1-test, 2-validation
train_df = df[df['partition'] == 0]
test_df = df[df['partition'] == 1]
val_df = df[df['partition'] == 2]

In [ ]:
# randomly select a fraction of images from the train dataframe
train_df = train_df(frac=0.1)
train_df.head()

In [ ]:
# similarly select  fraction of images from the validation dataframe
val_df = val_df(frac=0.1)
val_df.head()

In [ ]:
# Data generator class - this is a key class that is used to batch the data so as to
# reduce compute time as well as to fit training segments into available memory
# Additionally it allows you to specify multiple targets for classification
# Also allows for image cropping

class DataGenerator(tf.keras.utils.Sequence):

    def __init__(self, df, batch_size=32, dim=(64,64), n_channels=3, n_classes=?, shuffle=True):
        self.dim = dim
        self.batch_size = batch_size
        self.df = df
        self.n_channels = n_channels
        self.n_classes = n_classes
        self.shuffle = shuffle
        self.on_epoch_end()

    def __len__(self):
        return int(np.floor(len(self.df) / self.batch_size))

    def __getitem__(self, index):
        indexes = self.indexes[index*self.batch_size:(index+1)*self.batch_size]
        # saves memory by batching
        df_temp = self.df.iloc[indexes]
        X, y = self.__data_generation(df_temp)
        return X, y

    def on_epoch_end(self):
        self.indexes = np.arange(len(self.df))
        if self.shuffle == True:
            np.random.shuffle(self.indexes)
    
    def __data_generation(self, df_temp):
        X = np.empty((self.batch_size, *self.dim, self.n_channels))
        y = np.empty((self.batch_size, self.n_classes), dtype=int)
        for i, row in enumerate(df_temp.values):
            img = image.load_img('celeba-dataset/img_align_celeba/img_align_celeba/' + row[0]) # extract image
            #crop the iamges using the bounding boxes
          
            #normalize the cropped and resized images

            #specify the multiple targets now into your y vector

      
        return X, y                                                         

In [ ]:
# using vgg16 as feature extractor - this is a simpler, but equally eefective
# version of the code used for the Dogs vs Cats example
vgg16 = tf.keras.applications.VGG16(input_shape=(64, 64, 3), include_top=False, weights='imagenet')
vgg16.trainable = False

In [ ]:
# creating the model
model = tf.keras.Sequential([
    vgg16,
    # flatten the top 15 layers into one single layer containing the weights learned
    # in the first 15 layers of vgg16
    
    # add a dense layer for classification 
    
    
    # now specify the number of neurons in the output layer with the appropriate number of output neurons
    
])

In [ ]:
# compiling the model
model.compile(optimizer=SGD(learning_rate=0.0001), loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
# creating the train, test and validation data generators
train_generator = DataGenerator(train_df, batch_size=32, dim=(64,64), n_channels=3, n_classes=?, shuffle=True)
test_generator = DataGenerator(test_df, batch_size=32, dim=(64,64), n_channels=3, n_classes=?, shuffle=True)
val_generator = DataGenerator(val_df, batch_size=32, dim=(64,64), n_channels=3, n_classes=?, shuffle=True)

In [ ]:
# training the model
history = model.fit(train_generator, epochs=?, validation_data=val_generator)

In [ ]:
# plotting the training and validation accuracy
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Model Accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Val'], loc='upper left')
plt.show()

In [ ]:
# pre-process the landmarks file and compute mouth width

# normalize 

#bin

In [ ]:
# integrate the new version of the lanmarks data frame with the annotations, bounding box and partitions 
# data frames, just as you did before



In [ ]:
# modify your Data Generator class to handle your new targets